# MNIST shallow neural net with Keras

#### Load dependencies and make reproducible

In [1]:
import numpy as np
np.random.seed(1908)

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import TensorBoard

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Load MNIST data

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train.shape

(60000, 28, 28)

In [4]:
y_train[0:99]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0,
       9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9,
       3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0, 4, 5,
       6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9,
       0, 4, 6, 7, 4, 6, 8, 0, 7, 8, 3], dtype=uint8)

In [5]:
X_test[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  

In [6]:
y_train[0]

5

In [7]:
y_test.shape

(10000,)

In [8]:
X_test.shape

(10000, 28, 28)

#### Basic Preprocessing
Flatten the input and then normalise it

In [9]:
X_train = X_train.reshape(60000, 28 * 28 * 1).astype('float32')
X_test = X_test.reshape(10000, 28 * 28 * 1).astype('float32')

In [10]:
X_train /= 255
X_test /= 255

In [11]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

#### One-hot encode the output

In [12]:
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

In [13]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

#### Neural Network architecture

`Sequential` is the most used object on Keras and it is just a linear stack of layers.

When adding layers to the model, the first layer must always contain the `input_shape` info. You can also specify how you set the initial weights with the `kernel_intializer` argument.

In [14]:
model = Sequential()

model.add(Dense(64, activation = 'relu', input_shape = (28 * 28 * 1,),kernel_initializer='glorot_uniform'))

model.add(Dense((10), activation = 'softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


#### Configure model

To configure a model you use `compile()` to define the learning process. It requires an `optimizer`, a `loss` function and a list of metrics to evaluate its "success".

In [16]:
model.compile(loss = 'mean_squared_error', optimizer = SGD(lr=0.01), metrics = ['accuracy'])

#### Tensorboard

In [17]:
tensorboard = TensorBoard("logs/01_basic-mnist")

#### Train!

To train a model you use `fit()`. It is very valuable to define the `validation_split`- a fraction of the training data to be used as validation data, so he won't train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.

In [18]:
model.fit(X_train, y_train, batch_size = 128, epochs = 10, verbose = 1, 
          validation_split = 0.1, callbacks=[tensorboard])

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 1s 25us/step - loss: 0.0901 - acc: 0.0946 - val_loss: 0.0892 - val_acc: 0.1275
Epoch 2/10
54000/54000 [==============================] - 1s 23us/step - loss: 0.0887 - acc: 0.1507 - val_loss: 0.0877 - val_acc: 0.1955
Epoch 3/10
54000/54000 [==============================] - 1s 23us/step - loss: 0.0872 - acc: 0.2284 - val_loss: 0.0861 - val_acc: 0.2747
Epoch 4/10
54000/54000 [==============================] - 1s 23us/step - loss: 0.0856 - acc: 0.3036 - val_loss: 0.0844 - val_acc: 0.3473
Epoch 5/10
54000/54000 [==============================] - 1s 24us/step - loss: 0.0838 - acc: 0.3627 - val_loss: 0.0824 - val_acc: 0.3928
Epoch 6/10
54000/54000 [==============================] - 1s 23us/step - loss: 0.0818 - acc: 0.4061 - val_loss: 0.0802 - val_acc: 0.4362
Epoch 7/10
54000/54000 [==============================] - 1s 24us/step - loss: 0.0796 - acc: 0.4490 - val_loss: 0.0778 - val_acc:

#### Test Predictions

In [19]:
predictions = model.predict_classes(X_test, verbose = 2)
print(predictions)

[7 2 1 ... 9 8 6]


#### Test Final Accuracy

In [20]:
final_loss, final_acc = model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 0s 39us/step
Final loss: 0.0697, final accuracy: 0.5818
